In [3]:
val rawblocks = sc.textFile("linkage")

In [5]:
rawblocks.first

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"